In [1]:
import warnings
warnings.filterwarnings("ignore")
import gmsh
import meshio
import numpy as np
gmsh.initialize()

In [2]:
gmsh.model.add("3D")
L, B, H = 10, 10, 10
r = 1
channel = gmsh.model.occ.addBox(0, 0, 0, L, B, H)
resolution = 0.1

In [3]:
sphere = gmsh.model.occ.addSphere(5, 5, 5, r)

In [4]:
fluid = gmsh.model.occ.cut([(3, channel)], [(3, sphere)])

In [5]:
gmsh.model.occ.synchronize()
volumes = gmsh.model.getEntities(dim=3)

In [6]:
marker = 11
gmsh.model.addPhysicalGroup(volumes[0][0], [volumes[0][1]], marker)
gmsh.model.setPhysicalName(volumes[0][0], marker, "conductor")

In [7]:
surfaces = gmsh.model.occ.getEntities(dim=2)
left_marker = 1
right_marker = 3
sphere_marker = 5
spheres = []
walls = []
for surface in surfaces:
    com = gmsh.model.occ.getCenterOfMass(surface[0], surface[1])
    if np.allclose(com, [0, B/2, H/2]):
        gmsh.model.addPhysicalGroup(surface[0], [surface[1]], left_marker)
        left = surface[1]
        gmsh.model.setPhysicalName(surface[0], left_marker, "left")
    elif np.allclose(com, [L, B/2, H/2]):
        gmsh.model.addPhysicalGroup(surface[0], [surface[1]], right_marker)
        gmsh.model.setPhysicalName(surface[0], right_marker, "right")
        right = surface[1]
    elif np.isclose(com[2], 0) or np.isclose(com[1], B) or np.isclose(com[2], H) or np.isclose(com[1],0):
        walls.append(surface[1])
    else:
        spheres.append(surface[1])
gmsh.model.addPhysicalGroup(2, spheres, sphere_marker)
gmsh.model.setPhysicalName(2, sphere_marker, "sphere")

gmsh.model.mesh.field.add("Distance", 1)
gmsh.model.mesh.field.setNumbers(1, "FacesList", spheres)

gmsh.model.mesh.field.add("Threshold", 2)
gmsh.model.mesh.field.setNumber(2, "IField", 1)
gmsh.model.mesh.field.setNumber(2, "LcMin", resolution)
gmsh.model.mesh.field.setNumber(2, "LcMax", 20*resolution)
gmsh.model.mesh.field.setNumber(2, "DistMin", 0.5*r)
gmsh.model.mesh.field.setNumber(2, "DistMax", r)

gmsh.model.mesh.field.add("Distance", 3)
gmsh.model.mesh.field.setNumbers(3, "FacesList", [left])
gmsh.model.mesh.field.add("Threshold", 4)
gmsh.model.mesh.field.setNumber(4, "IField", 3)
gmsh.model.mesh.field.setNumber(4, "LcMin", 5*resolution)
gmsh.model.mesh.field.setNumber(4, "LcMax", 10*resolution)
gmsh.model.mesh.field.setNumber(4, "DistMin", 0.1)
gmsh.model.mesh.field.setNumber(4, "DistMax", 0.5)

gmsh.model.mesh.field.add("Min", 5)
gmsh.model.mesh.field.setNumbers(5, "FieldsList", [2, 4])
gmsh.model.mesh.field.setAsBackgroundMesh(5)

In [8]:
gmsh.model.occ.synchronize()
gmsh.model.mesh.generate(3)

In [9]:
gmsh.write("mesh3D.msh")

In [10]:
def create_mesh(mesh, cell_type, prune_z=False):
    cells = mesh.get_cells_type(cell_type)
    cell_data = mesh.get_cell_data("gmsh:physical", cell_type)
    points = mesh.points[:,:2] if prune_z else mesh.points
    out_mesh = meshio.Mesh(points=points, cells={cell_type: cells}, cell_data={"name_to_read":[cell_data]})
    return out_mesh

In [11]:
mesh_3d = meshio.read("mesh3D.msh")
tetrahedral_mesh = create_mesh(mesh_3d, "tetra")
meshio.write("mesh/10_0_10/mesh_tetr.xdmf", tetrahedral_mesh)

In [12]:
cmd_3d = 'mpirun -n 2 python3 ion_transport.py --working_dir=/home/lesh/dev/ssb/ --grid_info=10_0_10 --file_shape=10_10'
!{cmd_3d}

done loading tetrahedral mesh
done loading tetrahedral mesh
setting problem..
setting problem..
solving problem..
solving problem..
